# get aging

This is a text cell. Start editing!

In [1]:
val runtime = Runtime.getRuntime
val maxMemory = runtime.maxMemory / 1024 / 1024  // Convert bytes to MB
println(s"Max memory: $maxMemory MB")

Max memory: 36864 MB


In [2]:
import org.apache.spark.{io => spark_io}
import org.apache.spark.sql.types._
import scala.reflect.runtime.universe._
import org.apache.spark.storage.StorageLevel
import org.apache.spark.rdd._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._
import group.research.aging.spark.extensions._
import group.research.aging.spark.extensions.functions._
import kernel.display.html

In [3]:
import better.files._
import java.io.{File => JFile}

In [4]:
val data = File("/data")
val s2orc = data / "papers" / "s2orc" 
val all_pubmed_file = s2orc / "all_pubmed.parquet"
val all_aging_pubmed_file = s2orc / "all_aging_pubmed.parquet"

In [15]:
val all_pubmed = spark.read.parquet(all_pubmed_file.pathAsString)
all_pubmed.show(10,100)

+---------+--------------------+-----------------------------+----------------------------------------------------------------------------------------------------+----------------------------+----------------------------------------+----------------------+---------------+-----------------+-----------------------------+-----------------------------+---------------+------------------+-------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------

Aging paper

In [20]:
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions.col

def filterContent(df: DataFrame, terms: List[String]): DataFrame = {
  val regex = terms.map(term => s"\\b$term\\b").mkString("(", "|", ")")
  df.filter(col("content_text").rlike("(?i)" + regex))  // Added (?i) for case-insensitive matching
}

In [21]:
val terms_aging = List("aging", "ageing", "lifespan extension", "healthspan", "longevity")
val all_aging = filterContent(all_pubmed, terms_aging)
all_aging.writeParquet(all_aging_pubmed_file.pathAsString)

parts of /data/papers/s2orc/all_aging_pubmed.parquet merged!


/data/papers/s2orc/all_aging_pubmed.parquet

Test

In [24]:
val all_aging_df = spark.read.parquet(all_aging_pubmed_file.pathAsString)
all_aging_df.count()

755784

Moskalevs papers

In [16]:
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions.array_contains

def filterByAuthorLastName(df: DataFrame, author: String): DataFrame = {
  df.filter(array_contains(df("annotations_authorlastname"), author))
}
def filterByAuthorFirstName(df: DataFrame, author: String): DataFrame = {
  df.filter(array_contains(df("annotations_authorfirstname"), author))
}

In [26]:
val all_moskalev_pubmed_file = s2orc / "all_moskalev_pubmed.parquet"
val moskalev = filterByAuthorLastName(all_aging_df, "Moskalev")
moskalev.writeParquet(all_moskalev_pubmed_file.pathAsString)

parts of /data/papers/s2orc/all_moskalev_pubmed.parquet merged!


/data/papers/s2orc/all_moskalev_pubmed.parquet

In [27]:
val moskalev_loaded = spark.read.parquet(all_moskalev_pubmed_file.pathAsString)
moskalev_loaded.count()

102

In [30]:
moskalev_loaded.printSchema()

root
 |-- corpusid: long (nullable = true)
 |-- updated: string (nullable = true)
 |-- content_source_oainfo_license: string (nullable = true)
 |-- content_source_oainfo_openaccessurl: string (nullable = true)
 |-- content_source_oainfo_status: string (nullable = true)
 |-- content_source_pdfsha: string (nullable = true)
 |-- content_source_pdfurls: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- externalids_acl: string (nullable = true)
 |-- externalids_arxiv: string (nullable = true)
 |-- externalids_dblp: string (nullable = true)
 |-- externalids_doi: string (nullable = true)
 |-- externalids_mag: string (nullable = true)
 |-- externalids_pubmed: string (nullable = true)
 |-- externalids_pubmedcentral: string (nullable = true)
 |-- content_text: string (nullable = true)
 |-- annotations_abstract: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- annotations_author: array (nullable = true)
 |    |-- element: string (containsNull

In [28]:
filterByAuthorFirstName(moskalev_loaded, "Alexey").writeParquet((s2orc / "moskalev_pubmed.parquet").pathAsString)

parts of /data/papers/s2orc/moskalev_pubmed.parquet merged!


/data/papers/s2orc/moskalev_pubmed.parquet

In [29]:
val all_tacutu_pubmed_file = s2orc / "all_tacutu_pubmed.parquet"
val tacutu = filterByAuthorLastName(all_aging_df, "Tacutu")
tacutu.writeParquet(all_tacutu_pubmed_file.pathAsString)

org.apache.spark.sql.AnalysisException: [PATH_ALREADY_EXISTS] Path file:/data/papers/s2orc/all_tacutu_pubmed.parquet already exists. Set mode as "overwrite" to overwrite the existing path.